In [40]:
print(__doc__)
               #importing all the neccessary libraries

import numpy as np
import scipy as sp
import pandas as pd
from subprocess import check_output

import time, json
from datetime import date

import time
import math
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

Automatically created module for IPython interactive environment


In [67]:
df = pd.read_csv("final_merged.csv")


In [68]:
df = df.dropna()

In [69]:
uniqueVals = df["Code"].unique()  
len(uniqueVals)

442

In [70]:
df=df.set_index("Code") #for loc function, creted index

In [71]:
import time
import warnings
import numpy as np
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import matplotlib.pyplot as plt

In [72]:
def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

def load_data(df_name, seq_len, normalise_window):

    sequence_length = seq_len
    result = []
    for index in range(len(df_name) - sequence_length):
        result.append(df_name[index: index + sequence_length])
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)

    row = round(0.8 * result.shape[0]) #train and test data creation
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

    return [x_train, y_train, x_test, y_test]

def build_model():
    model = Sequential()

    model.add(LSTM(
    input_dim=1,
    output_dim=100,
    return_sequences=True))
    model.add(Dropout(0.2))

    model.add(Dense(
        output_dim=1))
    model.add(Activation('linear'))

    start = time.time()
    model.compile(loss='mse', optimizer='rmsprop')
    print 'compilation time : ', time.time() - start
    
    return model

def predict_sequences_multiple(model, data, window_size, prediction_len):
    #Predict sequence of 50 steps before shifting prediction run forward by 50 steps
    prediction_seqs = []
    for i in xrange(len(data)/prediction_len):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in xrange(prediction_len):
            predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

def plot_results_multiple(predicted_data, true_data, prediction_len):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in xrange(i * prediction_len)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()

In [74]:
#LSTM 2ND METHOD
for company in uniqueVals[:1]:  #for first 5 company
    #print company
    df1=(df.loc[company,:]).reset_index()
    #prices = df1['Close'] 
    prices = df1['Close'].values.astype('float32') 
    # Obtaining the values of closing data each day and converting to float as Neural network can easily model float 
    prices = prices.reshape(len(prices), 1)
    #Step 1 Load Data
    X_train, y_train, X_test, y_test = load_data(prices, 10, True)
    #step 2 build a model
    model = build_model()
    #Step 3 Train the model
    model.fit(
        X_train,
        y_train,
        batch_size=512,
        nb_epoch=10,
        validation_split=0.05)
    #step 4 predict and plot

    predictions = model.predict(X_test)
    #predictions = predict_sequences_multiple(model, X_test, 50, 50)
    predictions = np.reshape(predictions, (predictions.size,))
    #plot_results_multiple(predictions, y_test, 50)
    print predictions
    

/Users/ambikabohra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:39: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/Users/ambikabohra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 1))`


compilation time :  0.0660481452942


/Users/ambikabohra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`


ValueError: Error when checking target: expected activation_10 to have 3 dimensions, but got array with shape (118, 1)

In [61]:
#ARIMA
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
import seaborn as sns

/Users/ambikabohra/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [62]:


for company in uniqueVals[:1]:  #for first 5 company
    df1=(df.loc[company,:]).reset_index()
    df1['Price'] = df1['Close']
    df1['Date'] = df1['Date'].map(lambda x: str(x)[:7])
    Quantity_date = df1[['Price','Date']]
    Quantity_date_count=Quantity_date.groupby(['Date'])['Price'].aggregate('mean').reset_index().sort_values(by='Date', ascending=0)
    date=list(Quantity_date_count['Date'])
    quantity=list(Quantity_date_count['Price'])
    date_quantity = pd.DataFrame({'dates': date, 'quantity':quantity})
    date_quantity.index = date_quantity['dates'].map(lambda x: parser(x))
    date_quantity['quantity'] = date_quantity['quantity'].map(lambda x: float(x))
    
    #autocorelation 
    autocorrelation_plot(date_quantity)
    plt.show()
    
    #plot
    date_quantity.plot()
    plt.show()
    
    quantity = date_quantity.values
    size = int(len(quantity) * 0.66)
    train, test = quantity[0:size], quantity[size:len(quantity)]
    
    history = [x for x in train]
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=(1,1 ,0))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat[0])
        obs = test[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
    error = math.sqrt(mean_squared_error(test, predictions))
    print('Test MSE: %.3f' % error)
    
    
    #plot result
    plt.plot(test,color='blue' )
    plt.plot(predictions, color='red')
    plt.show()

ValueError: time data '2015100' does not match format '%Y-%m'

In [59]:
def parser(x):
    return datetime.strptime(x, '%Y-%m')